## Importando librerías

In [2]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime
from scipy.stats import truncnorm


## Función 'days_between' para calcular el la diferencia entre 2 fechas, recibe como parámetros:
- d1: fecha mínima
- d2: fecha máxima

retorna:
- un 'int' que se refiere a la diferencia en días entre las dos fechas

In [3]:
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return (d2 - d1).days

## Función 'get_count' para calcular los valores posibles en caso de columnas numéricas, categóricas o dates, recibe como parámetros:
- col: la columna de la cual queremos calcular el total de valores

retorna:
- un 'int' el cual se refiere a la cantidad total de valores posibles

In [64]:
def get_count(col):
    if col['type'] == 'category':
        return len(col['values'])
    elif col['type'] == 'numeric':
        return col['values']['max'] - col['values']['min']
    elif col['type'] == 'date':
        return days_between(col['max'], col['min'])
    else:
        return None

## Función 'getRows' para calcular el número máximo de registros que tendrán el dataframe (Este función y las anteriores son implementadas o usadas solamente en los casos en los que la columna 'random' de cualquier configuración de dataframe sea False), recibe como parámetros:
- conf: la configuración del dataframe del cual queremos obtener la cantidad máxima de registros

retorna:
- un 'int' que corresponde al número mayor entre todos los posibles valores de cada columna

In [5]:
def getRows(conf):
    columns_total_values = [
        get_count(col) for col in conf['columns'] if get_count(col) is not None
    ]

    return max(columns_total_values)

## Función 'generate_unique_id' para crear un str de 16 caracteres de manera aleatoria
retorna:
- un 'str' el cual es un string de 16 caracteres, que pueden ser letras y números

In [54]:
def generate_unique_id():
    caracteres = string.ascii_letters + string.digits
    unique_id = ''.join(random.choices(caracteres, k=16))
    return unique_id

## Función 'generate_unique_ids' para crear 'n' str de 16 caracteres, donde 'n' es el número de registros máximos que tendrá el dataframe dependiendo de la configuración, recibe como parámetros:
- count: el número total de 'ids' que necesitamos

retorna:
- una 'list' que contiene todos los 'ids' distintos

In [7]:
def generate_unique_ids(count):
    unique_ids = set()
    while len(unique_ids) < count:
        unique_ids.add(generate_unique_id())
    return list(unique_ids)


## Función 'generate_truncated_normal_data' para calcular valores de columnas numéricas en caso de que exista media y desviación, recibe como parámetros:
- diccionario que contiene los valores estadísticos
- cantidad de valores que deseamos

retorna:
- un objeto 'list' que contiene todos los valores calculados a partir de la información estadística proporcionada

In [61]:
def generate_truncated_normal_data(values, size):
    values = dict(values)
    
    for key, value in values.items():
        if value is None:
            raise TypeError('Se esperaban valores numéricos')
    
    min_val = float(values['min'])
    max_val = float(values['max'])
    mean = float(values['mean'])
    std = float(values['std'])

    # Calcular los parámetros de la distribución normal truncada
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)
    return data

## Función 'generate_col' para calcular cada una de las columnas de cualquier dataframe, esta función recibe como parámetros:
- col: se refiere a la configuración de la columna que deseamos crear.
- conf_list: es la lista de las configuraciones de dataframe, nos servirá para utilizar recursividad, ya que se necesitará para los casos donde existan llave foráneas.
- rows: se refiere al número de valores totales que tendrá nuestra columna.
- df_dict: este es un diccionario de dataframes. En los casos que existan llaves foráneas, nos servirá para verificar que el dataframe al que se hace referencia, ya ha sido creado, en caso contrario, lo crearemos y lo agregaremos a 'df_dict'

retorna: 
- un objeto 'Series' de pandas, el cual es la columna que deseabamos generar

In [62]:
def generate_col(col, conf_list, rows, df_dict):
    if col['type'] == "category":
        return random.choices(col['values'], k=rows)
    elif col['type'] == "numeric":
        if 'mean' in col['values'] and 'std' in col['values']:
            return generate_truncated_normal_data(col['values'], rows)
        return np.random.randint(col['values']['min'], col['values']['max'] + 1, size=rows)
    elif col['type'] == "unique":
        return generate_unique_ids(rows)
    elif col['type'] == 'foreign':
        ds_name, col_name_ref = col['values'].split('.')
        if ds_name in df_dict:
            return random.choices(df_dict[ds_name][col_name_ref], k=rows)
        else:
            temp_conf = [c for c in conf_list if c['ds'] == ds_name]
            temp_df = build_dataframe(temp_conf[0], conf_list, df_dict)
            df_dict[ds_name] = temp_df
            return random.choices(df_dict[ds_name][col_name_ref], k=rows)
    else:
        raise ValueError("Tipo de columna no válida")

## Función 'build_dataframe' se utiliza para crear únicamente 1 dataframe, recibe como parámetros:
- conf: la configuración completa del dataframe a crear
- conf_list: la lista de configuraciones, servirá para enviar las configuraciones a 'fun' para los casos recursivos o de llave foránea.
- df_dict: enviamos la diccionarios de diccionarios para ser utilizado en 'fun' 

retorna:
- un objeto 'DataFrame' de pandas

In [49]:
def build_dataframe(conf, conf_list, df_dict):
    rows = conf.get('random_rows') if conf['random'] else getRows(conf)
    df = pd.DataFrame()

    for col in conf['columns']:
        df[col['name']] = generate_col(col, conf_list, rows, df_dict)
    
    return df

## Función 'build_dataframes' sirve para crear la lista funal de dataframes a partir de una lista de configuraciones, recibe solamente un parámetro:
- conf_list: lista de configuraciones de los dataframes a crear.

retorna:
- una 'list' que contiene los diccionarios generados a partir de las configuraciones proporcionadas

In [37]:
def build_dataframes(conf_list):
    df_dict = {}
    result = []

    for conf in conf_list:
        df = build_dataframe(conf, conf_list, df_dict)
        df_dict[conf['ds']] = df
        result.append(df)

    # list(df_dict.values())
    return result

## Ejemplo de uso

In [68]:
d1 = {
        "ds": "dataset1",
        "columns": [
            {
                "name": "area",
                "type": "category",
                "values": ["TI", "FIN", "HR"]
            },
            {
                "name": "id",
                "type": "unique"
            },
            {
                'name': 'age',
                'type': 'numeric',
                'values': {'min': 5, 'max':15}
            }
        ],
        "random": False
    }


d2 = {
        "ds": "dataset2",
        "columns": [
            {
                "name": "id",
                "type": "unique"
            },
            {
                "name": "area",
                "type": "foreign",
                "values": "dataset1.area"
            },
            {
                "name": "subarea",
                "type": "category",
                "values": ["SA1", "SA2", "SA3", "SA4"]
            }
        ],
        "random": False
    }


d3 = {
        "ds": "dataset3",
        "columns": [
            {
                "name": "id",
                "type": "unique"
            },
            {
                "name": "subarea",
                "type": "foreign",
                "values": "dataset2.subarea"
            },
            {
                "name": "income",
                "type": "numeric",
                "values": {"min": 1, "max": 10}
            },
            {
                "name": "goal",
                "type": "numeric",
                "values": {"min": 1582.0, "max": 2042.666667, "std": 605.884753,
                "mean": 2349}
            }
            ],
        "random": True,
        "random_rows": 100
    }

configuration_list = [d2, d3, d1]

dataframe_list = build_dataframes(configuration_list)

dataframe_list


[                 id area subarea
 0  Q5tNoYylu5hz92Oa   HR     SA3
 1  ZEQ4sR3HamC0keVD   HR     SA1
 2  Yt2ONYzq4BN5rnkN  FIN     SA3
 3  RO3eQxzAt6m4S4Xc  FIN     SA4,
                   id subarea  income         goal
 0   4ZAgLZG6DX2Pg6aj     SA3       1  1710.152826
 1   q5fVdL93rdqz7bV6     SA3      10  1797.885372
 2   6OW07kCYtg6tbY0I     SA1       6  1666.560907
 3   DdLqhmBd3TSLGRYU     SA1       6  1942.661916
 4   6Um3jmbp4X3SFdwV     SA4       5  1972.050076
 ..               ...     ...     ...          ...
 95  i41ngaUrkfoXtmac     SA3       8  1694.970959
 96  Ct4YiydsKm7l59EG     SA3      10  1979.214321
 97  LIeTLXL31ydvzGOQ     SA3      10  1915.268247
 98  FDX4E4erqKIgzeAn     SA3       3  1794.796381
 99  k7iiFPOZ29aD8uM2     SA4       4  1930.856301
 
 [100 rows x 4 columns],
   area                id  age
 0   TI  9iAPbToECZ4oEcFv    5
 1  FIN  y41AGZrKEhP9bK61    9
 2  FIN  UNMT5uQfFYDWrYLt    9
 3   TI  LwUtdWvflqQu6LoA    7
 4  FIN  LkFFSxPlyROKXNz2    9
 5  